# Day 5: Supply Stacks

# Part 1

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked `crates`, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a `giant cargo crane` capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```
In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
 ```
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:
```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
 ```
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
 ```

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

**After the rearrangement procedure completes, what crate ends up on top of each stack?**



In [1]:
%load_ext nb_mypy

Version 1.0.4


In [6]:
from typing import Generator

from pathlib import Path

INPUT_FILE = Path.cwd() / "inputs" / "day5.txt"
assert INPUT_FILE.exists(), INPUT_FILE

def ingest_input(input_file: Path) -> Generator[str | list, None, None]:
    starting_stacks: list[str] = []
    move_section: bool = False

    with open(input_file) as f_in:
    
        while not move_section:
            line = next(f_in)
            if not line:
                yield starting_stacks
                move_section = True
            starting_stacks.append(line)
        while True:
            try:
                line = next(f_in)
                yield line
            except StopIteration:
                break

for i, line in zip(range(15), ingest_input(INPUT_FILE)):
    print(i, line)

RuntimeError: generator raised StopIteration

## Part Two
It seems like there is still quite a bit of duplicate work planned. Instead, the Elves would like to know the number of pairs that overlap at all.

In the above example, the first two pairs (2-4,6-8 and 2-3,4-5) don't overlap, while the remaining four pairs (5-7,7-9, 2-8,3-7, 6-6,4-6, and 2-6,4-8) do overlap:

5-7,7-9 overlaps in a single section, 7.
2-8,3-7 overlaps all of the sections 3 through 7.
6-6,4-6 overlaps in a single section, 6.
2-6,4-8 overlaps in sections 4, 5, and 6.
So, in this example, the number of overlapping assignment pairs is 4.

**In how many assignment pairs do the ranges overlap?**

In [3]:
def partial_overlap(r1: range, r2: range) -> bool:
    r1_set = set(r1)
    r2_set = set(r2)
    for range_, set_ in zip((r1, r2), (r1_set, r2_set)):
        set_.add(range_.stop)  # end of range should be included

    if r1_set.intersection(r2_set):
        return True
    return False


overlap_count = 0
for i, pairs in enumerate(ingest_range_pairs(INPUT_FILE), start=1):

    if partial_overlap(*pairs):
        overlap_count += 1
        # print(f"{i} Overlap {pairs}")

print(f"\n{overlap_count=}")


overlap_count=909


## Alternate Solution using custom objects

Would be more useful if you could do things like add and subtract intervals, but that's not needed for this exercise.

In [4]:
from __future__ import annotations

class Interval:
    _start: int
    _stop: int
    _range: range
    _set: set[int]

    def __init__(self, interval: str):
        if not isinstance(interval, str):
            raise TypeError(f"Expected interval string in the format '1-99', got {type(interval)}")

        start, stop = interval.split("-")
        self._start = int(start)
        self._stop = int(stop)
        self._range = range(self._start, self._stop + 1)
        self._set = set(self._range)

    def __repr__(self) -> str:
        return f"Interval({self._start}, {self._stop})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, self.__class__):
            return False
        return self._start == other._start and self._stop == other._stop

    def __contains__(self, other) -> bool:
        return self._set.issuperset(set(other)) 

    def __iter__(self):
        for i in self._range:
            yield i

    def intersection(self, other) -> set[int]:  # type: ignore[valid-type]
        return self._set.intersection(set(other))

In [5]:
a = Interval("1-10")
b = Interval("2-8")
A = Interval("1-10")
c = Interval("11-20")

assert a == A
assert a != object()
assert a != b
assert b in a
assert c not in a
assert [1, 2] in a

In [6]:
def ingest_intervals(input_file: Path) -> Generator[tuple[Interval, Interval], None, None]:
    with open(input_file) as f_in:
        for line in f_in:
            s1, s2 = line.rstrip().split(",")
            yield Interval(s1), Interval(s2)

In [7]:
full_overlap_count = 0
partial_overlap_count = 0
for i1, i2 in ingest_intervals(INPUT_FILE):

    if i1 in i2 or i2 in i1:
        full_overlap_count += 1

    if i1.intersection(i2):
        partial_overlap_count += 1

print(f"\n{full_overlap_count=}")
print(f"\n{partial_overlap_count=}")


full_overlap_count=518

partial_overlap_count=909
